In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep  2 10:39:59 2020

@author: c3121
"""


#%%
import tensorflow as tf
import os
import keras
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
def apply_color_mask(hsv,img,low,high):
    # Apply color mask to image
    mask = cv2.inRange(hsv, low, high)
    res = cv2.bitwise_and(img,img, mask= mask)
    return res

#%%
base_path = r'health'#這裡要改
img_0=[]
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".JPG"):
            filename = os.path.join(root, file)
            file_size = os.path.getsize(filename)
            category_name = os.path.basename(root)
            if file_size >= 2048:
                #print(filename)
                im = cv2.imread(filename)
                #im = cv2.imread(r'diease/c47ccf21-5249-4a95-8d33-0d640e3529f7___RS_L.Scorch 0158.JPG')
                im=cv2.resize(im,(128,128), interpolation = cv2.INTER_CUBIC)
                img_0.append(im)
img_0_array=np.asarray(img_0)
img_0_label=np.ones(img_0_array.shape[0])*0 
base_path = r'diease'#這裡要改
img_1=[]
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".JPG"):
            filename = os.path.join(root, file)
            file_size = os.path.getsize(filename)
            category_name = os.path.basename(root)
            #print(filename)
            if file_size >= 2048:
                #print(filename)
                im = cv2.imread(filename)
                #print(filename)
                #im = cv2.imread(r'diease/c47ccf21-5249-4a95-8d33-0d640e3529f7___RS_L.Scorch 0158.JPG')
                im=cv2.resize(im,(128,128), interpolation = cv2.INTER_CUBIC)
                img_1.append(im)
img_1_array=np.asarray(img_1)
img_1_label=np.ones(img_1_array.shape[0])*1
img_array=np.concatenate((img_0_array,img_1_array) ,axis=0)
img_label=np.concatenate((img_0_label,img_1_label),axis=0)
img_label = keras.utils.to_categorical(img_label, num_classes =2)
#print(img_label)
import random
temp_img = list(zip(img_array, img_label))
random.shuffle(temp_img)
img_array, img_label = zip(*temp_img)
img_array=np.asarray(img_array)
img_label=np.asarray(img_label)
print(img_label)
print(img_label[1500:1600])
from sklearn.model_selection import train_test_split
img_train,img_test,label_train,label_test=train_test_split(img_array, img_label, test_size=0.2,shuffle = True)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import RMSprop, SGD
from keras.layers.normalization import BatchNormalization
model = Sequential()
  

 #First Convolution and Pooling layer
model.add(Conv2D(96,(11,11),strides=(4,4),input_shape=(128,128,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
  

  
    #Second Convolution and Pooling layer
model.add(Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    

    #Three Convolution layer and Pooling Layer
model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    

    #Fully connection layer
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.5))
   
    #Classfication layer
model.add(Dense(2,activation='softmax'))
model.summary()
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'

model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
print(len(img_train))

In [ ]:
history=model.fit(img_train, label_train, batch_size=64, epochs=100,shuffle=True, validation_split=0.2)

In [ ]:
n=0
for index in range(len(img_test)):
    test_im=img_test[index]
   #plt.subplot(2,4,index+1)
    #plt.imshow(cv2.cvtColor(test_im, cv2.COLOR_BGR2RGB))
    test_im=test_im.reshape(-1,128,128,3)
    pred=model.predict(test_im)
    
    a=np.argmax(pred)
    b=np.argmax(label_test[index])
    if(a==b):
        n=n+1
    #plt.show()
print('準確:',n,'總共:' ,len(img_test))    

In [ ]:
#model.save('cnn.h5')

In [ ]:
model_fcnn = Sequential()
from keras.models import load_model   
#model_train=load_model('cnn.h5')
 #First Convolution and Pooling layer
model_fcnn.add(Conv2D(96,(3,3),strides=(2,2),input_shape=(128,128,3),padding='same',activation='relu'))
#model_fcnn.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'))
  

  
    #Second Convolution and Pooling layer
model_fcnn.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu'))
#model_fcnn.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'))
    

    #Three Convolution layer and Pooling Layer
model_fcnn.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu'))
#model_fcnn.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu'))
#model_fcnn.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu'))
#model_fcnn.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu'))
model_fcnn.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'))
#model_fcnn.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'))
    

    #Fully cnn layer
model_fcnn.add(Conv2DTranspose(32,(3,3),strides=(2,2),padding='same',activation='relu'))
model_fcnn.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'))
model_fcnn.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'))
model_fcnn.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'))
#model_fcnn.add(Conv2DTranspose(256,(3,3),strides=(2,2),padding='same',activation='relu'))
model_fcnn.add(Conv2D(3,(3,3),strides=(1,1),padding='same',activation='softmax'))
    
    #Classfication layer

model_fcnn.summary()
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'

model_fcnn.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
#img_1_array=img_1_array.astype('float32')/255
history=model_fcnn.fit(img_1_array,img_1_array , batch_size=32, epochs=50,shuffle=True, validation_split=0.2)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from keras.models import load_model   
#model_fcnn=load_model('fcnn.h5')
n=0
for im in img_1_array:
    #im = cv2.imread('diease/01cff44f-9564-42f7-9a29-3daa487b306a___RS_L.Scorch 1333.JPG')
    #im=cv2.resize(im,(128,128), interpolation = cv2.INTER_CUBIC)
    #im=im.astype('float32')/255
    plt.subplot(1,2,1)
    plt.imshow(cv2.cvtColor(im,cv2.COLOR_BGR2RGB))
    #plt.show()
    

    imOut=im.copy()
    im=im.astype('float32')/255

    im=im.reshape(-1,128,128,3)
    pred=model_fcnn.predict(im)

    #print(pred)
    #原本顏色太暗所以條遺下
    pred=pred.astype('float32')
    #plt.subplot(2,2,2)
    #plt.imshow(cv2.cvtColor(pred[0],cv2.COLOR_BGR2RGB))
    #mask=np.zeros((128,128),dtype=np.uint8)
    for x in  range(128):
        for y in range(128):
            if(pred[0][x][y][1]>0.3):   
                pred[0][x][y]=(0,0,0)
            else:
                imOut[x][y]=(0,0,255)
    plt.subplot(1,2,2)
    plt.imshow(cv2.cvtColor(imOut,cv2.COLOR_BGR2RGB))            
    #print(pred)
    #pred=pred.astype('float32')
    #pred=cv2.cvtColor(pred.reshape(128,128,3),cv2.COLOR_BGR2RGB)
    #plt.subplot(2,2,2)
    #plt.imshow(pred)
    #plt.subplot(2,2,3)
    #plt.imshow(mask,cmap='gray')
    #print(imOut.dtype)
    
    #ret,mask = cv2.threshold(mask,55,255,cv2.THRESH_BINARY)
    #print(mask.dtype)
    #im_res= cv2.bitwise_and(imOut,imOut, mask= mask)
    #plt.subplot(2,2,4)
    
    #plt.imshow(cv2.cvtColor(im_res,cv2.COLOR_BGR2RGB))       
    plt.show()
    #cv2.imwrite('cnn_diease/cnn_diease_{}.jpg'.format(n),im_res)
    n=n+1

In [ ]:
#model_fcnn.save('fcnn.h5')

In [ ]:
for im in img_1_array[:1]:
    # 建立 Selective Search 分割器
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

    # 設定要進行分割的圖形
    ss.setBaseImage(im)

    # 使用快速模式（精準度較差）
    #ss.switchToSelectiveSearchFast()

    # 使用精準模式（速度較慢）
    ss.switchToSelectiveSearchQuality()

    # 執行 Selective Search 分割
    rects = ss.process()

    print('候選區域總數量： {}'.format(len(rects)))

    # 要顯示的候選區域數量
    numShowRects = 50

    # 每次增加或減少顯示的候選區域數量
    increment = 50

    plt.subplot(1,2,1)
    plt.imshow(cv2.cvtColor(im,cv2.COLOR_BGR2RGB))
    #plt.show()
    


    im=im.astype('float32')/255

    im=im.reshape(-1,128,128,3)
    pred=model_fcnn.predict(im)

    #print(pred)
    #原本顏色太暗所以條遺下
    pred=pred.astype('float32')
    
    #for x in  range(128):
    #    for y in range(128):
    
    #        pred[0][x][y][1]=
    #print(pred)
    #pred=pred.astype('float32')
    pred=cv2.cvtColor(pred.reshape(128,128,3),cv2.COLOR_BGR2RGB)
    plt.subplot(1,2,2)
    plt.imshow(pred)
    plt.show()


  # 以迴圈處理每一個候選區域
    for i, rect in enumerate(rects):
      # 以方框標示候選區域

        if (i < numShowRects):
            imOut = im[0].copy()
            x, y, w, h = rect
            imrect=pred[y:y+h,x:x+w]
            print(imrect.shape)
            print(rect)
            imrect=cv2.resize(imrect,(128,128), interpolation = cv2.INTER_CUBIC)
            plt.subplot(1,2,1)
            plt.imshow(imrect)
            imrect=imrect.reshape(-1,128,128,3)
            pred=model.predict(imrect)
            if(np.argmax(pred)==0):

                cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 0, 1), 1, cv2.LINE_AA)
                imOut=imOut.astype('float32')
                plt.subplot(1,2,2)
                plt.imshow(cv2.cvtColor(imOut,cv2.COLOR_BGR2RGB))
            plt.show()
        else:
            break

In [ ]:
img_train = img_train.astype('float32')/255
img_test=img_test.astype('float32')/255

model_Conv2DTranspose = Sequential()
  

#test

model_Conv2DTranspose.add(Conv2D(96, kernel_size=(3, 3),strides=(2,2), activation='relu', kernel_initializer='he_normal',padding='same', input_shape=(128,128,3)))
#model_Conv2DTranspose.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal',padding='same'))
#model_Conv2DTranspose.add(Conv2D(16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal'))

#model_Conv2DTranspose.add(Conv2DTranspose(16, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
#model_Conv2DTranspose.add(Conv2DTranspose(32, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal',padding='same'))
model_Conv2DTranspose.add(Conv2DTranspose(3, kernel_size=(3,3),strides=(2,2), activation='relu', kernel_initializer='he_normal',padding='same'))

    
    
    
 #First Convolution and Pooling layer
#model_Conv2DTranspose.add(Conv2D(80,(3,3),strides=(2,2),input_shape=(128,128,3),padding='same',activation='relu', kernel_initializer='he_uniform'))
#model_Conv2DTranspose.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'))
  

  
    #Second Convolution and Pooling layer
#model_Conv2DTranspose.add(Conv2D(32,(3,3),strides=(2,2),padding='same',activation='relu', kernel_initializer='he_uniform'))
#model_Conv2DTranspose.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'))
    

    #Three Convolution layer and Pooling Layer
#model_Conv2DTranspose.add(Conv2D(16,(3,3),strides=(2,2),padding='same',activation='relu', kernel_initializer='he_uniform'))
#model_Conv2DTranspose.add(Conv2D(32,(3,3),strides=(1,1),activation='relu'))
#model_Conv2DTranspose.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'))
#model_Conv2DTranspose.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'))

#model_Conv2DTranspose.add(Conv2DTranspose(32, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))



#model_Conv2DTranspose.add(Conv2DTranspose(16,kernel_size=(3, 3), strides=(2,2), activation='relu',padding='same', kernel_initializer='he_uniform'))
#model_Conv2DTranspose.add(Conv2D(256, (3, 3), activation='relu',padding='same'))
#model_Conv2DTranspose.add(Conv2DTranspose(32,kernel_size=(3, 3),padding='same', strides=(2,2), activation='relu', kernel_initializer='he_uniform'))
#model_Conv2DTranspose.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
#model_Conv2DTranspose.add(Conv2DTranspose(64,kernel_size=(3, 3),padding='same', strides=(2,2), activation='relu', kernel_initializer='he_uniform'))
#model_Conv2DTranspose.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
#model_Conv2DTranspose.add(Conv2DTranspose(32,kernel_size=(3, 3),padding='same', strides=(2,2), activation='relu', kernel_initializer='he_uniform'))
#model_Conv2DTranspose.add(Conv2DTranspose(3,kernel_size=(3, 3), strides=(2,2), activation='sigmoid', kernel_initializer='he_uniform',padding='same'))

model_Conv2DTranspose.summary()
#weights = [np.asarray([[[[1]]]]), np.asarray([0])]
# store the weights in the model
#model_Conv2DTranspose.set_weights(weights)
model_Conv2DTranspose.compile(optimizer='adam', loss=objective, metrics=['accuracy'])
model_Conv2DTranspose.fit(img_train, img_train, batch_size=64, epochs=10,shuffle=True, validation_split=0.2)

In [ ]:
print((model_Conv2DTranspose.outputs))

In [ ]:
for im in img_1_array:
    #im = cv2.imread('diease/01cff44f-9564-42f7-9a29-3daa487b306a___RS_L.Scorch 1333.JPG')
    #im=cv2.resize(im,(128,128), interpolation = cv2.INTER_CUBIC)
    im=im.astype('float32')/255
    plt.subplot(1,2,1)
    plt.imshow(cv2.cvtColor(im,cv2.COLOR_BGR2RGB))
    #plt.show()
    
    im=im.reshape(-1,128,128,3)
    pred=model_Conv2DTranspose.predict(im)

    #print(pred)
    pred=pred.astype('float32')
    #for x in  range(128):
    #    for y in range(128):
    
    #        pred[0][x][y]=np.argmax(pred[0][x][y])
    #print(pred)
    #pred=pred.astype('float32')
    pred=cv2.cvtColor(pred.reshape(128,128,3),cv2.COLOR_BGR2RGB)
    plt.subplot(1,2,2)
    plt.imshow(pred)
    plt.show()